# Feedback Loop in RAG

### 피드백 루프의 필요성

- 기존 RAG 시스템의 한계는 다음과 같다:
- 검색 결과가 항상 동일한 유사도 기준에 따라 반환되어 문맥 변화나 사용자 의도 적응이 어렵다.
- 잘못된 정보가 반복될 가능성이 있으며 검증된 응답이 데이터베이스에 반영되지 않는다.
- 사용자의 만족도나 응답 효과성에 대한 학습이 없다.


### 피드백 루프의 주요 기능

**1. 효과가 있었던 것과 없었던 것을 기억:**
사용자가 높은 평가(예: thumbs up, 만족도 점수)를 준 Q&A 쌍은 양질의 지식 자산으로 기록된다.
부정확한 응답은 다시 학습하거나 제외할 수 있는 학습 대상으로 분류된다.

**2. 문서 연관성 점수 조정:**
문서나 청크에 대해 반복적으로 높은 평가를 받은 경우, 해당 청크의 연관성 점수를 동적으로 상향 조정한다.
반대로, 유사도는 높았지만 자주 부정적 피드백을 받은 청크는 패널티를 부여하여 검색 우선순위에서 낮춘다.

**3. 성공적인 Q&A 쌍을 지식 창고에 통합:**
특정 쿼리에 대해 우수한 응답이 생성되었다면, 해당 Q&A 쌍을 별도의 Retrieval 대상 지식창고로 저장할 수 있다.
이후 유사 질문이 들어오면, 벡터 검색보다 빠르게 해당 사례를 참조하여 응답 품질을 향상시킨다.

**4. 각 상호작용을 통해 더욱 스마트해짐:**
피드백은 단순 통계가 아니라 랭킹 학습, 임베딩 리트레이닝, 규칙 학습 등에 활용되어 시스템 전반의 성능을 개선한다.
반복적인 상호작용 속에서 사용자 맞춤화(Personalization)도 가능해진다.


### 예시 시나리오

사용자 A가 “퇴사 후 연차 정산 방법”에 대해 질문하고, 응답에 ‘매우 만족’을 선택함

→ 해당 질문과 응답은 ‘신뢰된 Q&A’로 등록

→ 이후 유사한 질문이 들어오면 해당 Q&A가 우선적으로 사용됨

→ 반대로 ‘잘못된 정책 정보’가 포함된 청크는 score가 낮아져 검색에서 제외됨

## Setting Up the Environment

In [1]:
import fitz
import numpy as np
import json
from datetime import datetime

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Extracting Text from a PDF File

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    PDF 파일에서 텍스트를 추출합니다.

    Args:
        pdf_path (str): PDF 파일 경로

    Returns:
        str: PDF에서 추출된 전체 텍스트
    """
    # PDF 파일 열기
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 전체 텍스트를 저장할 문자열 초기화

    # 각 페이지를 순회하며 텍스트 추출
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]               # 해당 페이지 가져오기
        text = page.get_text("text")         # 텍스트 형식으로 내용 추출
        all_text += text                     # 추출된 텍스트 누적

    # 추출된 전체 텍스트 반환
    return all_text

## Chunking the Extracted Text

In [4]:
def chunk_text(text, n=1000, overlap=200):
    """
    주어진 텍스트를 n자 단위로 분할하되, 각 청크 간에 overlap만큼 겹치게 합니다.

    Args:
        text (str): 분할할 텍스트
        n (int): 각 청크의 문자 수 (기본값: 1000)
        overlap (int): 청크 간 겹치는 문자 수 (기본값: 200)

    Returns:
        List[str]: 분할된 텍스트 청크 리스트
    """
    chunks = []  # 청크들을 저장할 리스트 초기화

    # n - overlap 만큼 이동하면서 청크 생성
    for i in range(0, len(text), n - overlap):
        # i에서 i + n까지의 텍스트 조각을 청크로 추가
        chunks.append(text[i:i + n])

    return chunks  # 생성된 청크 리스트 반환

## Simple Vector Store Implementation

In [5]:
class SimpleVectorStore:
    """
    NumPy를 활용한 간단한 벡터 저장소 구현체입니다.
    """
    def __init__(self):
        """
        벡터 저장소 초기화
        """
        self.vectors = []     # 임베딩 벡터 리스트
        self.texts = []       # 원본 텍스트 리스트
        self.metadata = []    # 각 텍스트의 메타데이터 리스트

    def add_item(self, text, embedding, metadata=None):
        """
        단일 텍스트 항목을 벡터 저장소에 추가합니다.

        Args:
            text (str): 원본 텍스트
            embedding (List[float]): 텍스트의 임베딩 벡터
            metadata (dict, optional): 추가 메타데이터 (기본값: 빈 딕셔너리)
        """
        self.vectors.append(np.array(embedding))  # 임베딩을 NumPy 배열로 변환하여 저장
        self.texts.append(text)                   # 원본 텍스트 저장
        self.metadata.append(metadata or {})      # 메타데이터 저장 (None일 경우 빈 딕셔너리)

    def similarity_search(self, query_embedding, k=5):
        """
        질의 임베딩과 가장 유사한 텍스트를 검색합니다.

        Args:
            query_embedding (List[float]): 질의 임베딩 벡터
            k (int): 반환할 상위 결과 개수

        Returns:
            List[Dict]: 유사한 항목 리스트 (텍스트, 메타데이터, 유사도 포함)
        """
        if not self.vectors:
            return []  # 저장된 벡터가 없으면 빈 리스트 반환

        # 질의 벡터를 NumPy 배열로 변환
        query_vector = np.array(query_embedding)

        # 각 벡터와의 코사인 유사도 계산
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = np.dot(query_vector, vector) / (
                np.linalg.norm(query_vector) * np.linalg.norm(vector)
            )
            similarities.append((i, similarity))  # 인덱스와 유사도 점수 저장

        # 유사도 기준 내림차순 정렬
        similarities.sort(key=lambda x: x[1], reverse=True)

        # 상위 k개 항목 반환
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadata[idx],
                "similarity": score
            })

        return results

## Creating Embeddings

In [6]:
def create_embeddings(text, model="text-embedding-3-small"):
    """
    주어진 텍스트에 대해 임베딩을 생성합니다.

    Args:
        text (str 또는 List[str]): 임베딩을 생성할 입력 텍스트(또는 텍스트 리스트)
        model (str): 사용할 임베딩 모델 이름

    Returns:
        List[float] 또는 List[List[float]]: 생성된 임베딩 벡터 또는 벡터 리스트
    """
    # 입력이 문자열 하나일 수도 있고, 문자열 리스트일 수도 있으므로 리스트 형태로 통일
    input_text = text if isinstance(text, list) else [text]

    # 지정된 모델을 사용하여 임베딩 생성 요청
    response = client.embeddings.create(
        model=model,
        input=input_text
    )

    # 입력이 단일 문자열이었을 경우, 첫 번째 임베딩만 반환
    if isinstance(text, str):
        return response.data[0].embedding

    # 여러 문자열일 경우, 모든 임베딩 리스트 반환
    return [item.embedding for item in response.data]

## Feedback System Functions

In [7]:
def get_user_feedback(query, response, relevance, quality, comments=""):
    """
    사용자 피드백을 딕셔너리 형태로 포맷합니다.
    
    Args:
        query (str): 사용자의 질의
        response (str): 시스템의 응답
        relevance (int): 관련성 점수 (1~5)
        quality (int): 품질 점수 (1~5)
        comments (str): 선택적인 코멘트 (기본값: 빈 문자열)
        
    Returns:
        Dict: 포맷된 피드백 딕셔너리
    """
    return {
        "query": query,                      # 사용자 질의
        "response": response,                # 시스템 응답
        "relevance": int(relevance),         # 관련성 평가 점수
        "quality": int(quality),             # 품질 평가 점수
        "comments": comments,                # 사용자 코멘트
        "timestamp": datetime.now().isoformat()  # 타임스탬프 (ISO 형식)
    }

In [8]:
def store_feedback(feedback, feedback_file="dataset/feedback_data.json"):
    """
    피드백을 JSON 파일에 저장합니다.
    
    Args:
        feedback (Dict): 피드백 데이터
        feedback_file (str): 피드백 파일 경로
    """
    with open(feedback_file, "a") as f:
        json.dump(feedback, f)
        f.write("\n")

In [9]:
def load_feedback_data(feedback_file="dataset/feedback_data.json"):
    """
    피드백 데이터를 JSON 파일에서 불러옵니다.

    Args:
        feedback_file (str): 피드백 파일 경로.

    Returns:
        List[Dict]: 피드백 항목으로 구성된 리스트.
    """
    feedback_data = []
    try:
        with open(feedback_file, "r") as f:
            for line in f:
                if line.strip():  # 빈 줄 제외
                    feedback_data.append(json.loads(line.strip()))
    except FileNotFoundError:
        print("피드백 데이터 파일이 존재하지 않습니다. 빈 리스트로 시작합니다.")

    return feedback_data

## Document Processing with Feedback Awareness

In [10]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    피드백 루프가 포함된 RAG(Retrieval Augmented Generation)용 문서 처리 함수.
    이 함수는 전체 문서 처리 파이프라인을 다룹니다:
    1. PDF에서 텍스트 추출
    2. 겹침이 있는 텍스트 청크 분할
    3. 각 청크에 대한 임베딩 생성
    4. 메타데이터와 함께 벡터 데이터베이스에 저장

    Args:
        pdf_path (str): 처리할 PDF 파일 경로
        chunk_size (int): 각 텍스트 청크의 문자 수
        chunk_overlap (int): 연속된 청크 간의 겹치는 문자 수

    Returns:
        Tuple[List[str], SimpleVectorStore]: 다음을 포함하는 튜플
            - 텍스트 청크 리스트
            - 임베딩 및 메타데이터가 저장된 벡터 저장소
    """
    # 1단계: PDF 문서에서 원시 텍스트 추출
    print("PDF에서 텍스트 추출 중... (Extracting text from PDF...)")
    extracted_text = extract_text_from_pdf(pdf_path)
    
    # 2단계: 더 나은 문맥 유지를 위해 텍스트를 겹치는 청크로 분할
    print("텍스트 청크 분할 중... (Chunking text...)")
    chunks = chunk_text(extracted_text, chunk_size, chunk_overlap)
    print(f"{len(chunks)}개의 텍스트 청크 생성됨 (Created {len(chunks)} text chunks)")
    
    # 3단계: 각 텍스트 청크에 대한 벡터 임베딩 생성
    print("청크에 대한 임베딩 생성 중... (Creating embeddings for chunks...)")
    chunk_embeddings = create_embeddings(chunks)
    
    # 4단계: 청크와 임베딩을 저장할 벡터 저장소 초기화
    store = SimpleVectorStore()
    
    # 5단계: 각 청크를 임베딩과 함께 벡터 저장소에 추가
    # 메타데이터에는 피드백 기반 개선을 위한 정보 포함
    for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
        store.add_item(
            text=chunk,
            embedding=embedding,
            metadata={
                "index": i,                # 원문 내 위치
                "source": pdf_path,        # 원본 문서 경로
                "relevance_score": 1.0,    # 초기 관련성 점수 (피드백으로 갱신 가능)
                "feedback_count": 0        # 해당 청크에 대한 피드백 횟수
            }
        )
    
    print(f"{len(chunks)}개의 청크가 벡터 저장소에 추가됨 (Added {len(chunks)} chunks to the vector store)")
    return chunks, store

## Relevance Adjustment Based on Feedback

In [11]:
def assess_feedback_relevance(query, doc_text, feedback):
    """
    이전 피드백 항목이 현재 쿼리 및 문서와 관련 있는지를 LLM을 사용하여 평가합니다.

    이 함수는 현재 쿼리, 과거 쿼리+피드백, 그리고 문서 내용을 LLM에 전달하여 
    과거 피드백이 현재 정보 검색에 영향을 줄 수 있는지를 판단합니다.

    Args:
        query (str): 현재 사용자의 정보 검색 쿼리
        doc_text (str): 평가 대상 문서의 텍스트 내용
        feedback (Dict): 'query'와 'response' 키를 포함하는 이전 피드백 데이터

    Returns:
        bool: 피드백이 현재 쿼리/문서와 관련 있다고 판단되면 True, 그렇지 않으면 False
    """
    # 시스템 프롬프트: LLM이 '관련 있음/없음' 판단만 하도록 지시
    system_prompt = """당신은 과거 피드백이 현재 쿼리 및 문서와 관련이 있는지를 판단하는 AI 시스템입니다.
    답변은 오직 'yes' 또는 'no'로만 하세요. 설명을 제공하지 말고, 오직 관련성 여부만 판단하십시오."""

    # 사용자 프롬프트: 현재 쿼리, 과거 피드백, 문서 내용을 포함
    user_prompt = f"""
    현재 쿼리(Current query): {query}
    피드백이 달렸던 과거 쿼리(Past query that received feedback): {feedback['query']}
    문서 내용(Document content): {doc_text[:500]}... [중략]
    피드백이 달린 응답(Past response that received feedback): {feedback['response'][:500]}... [중략]

    이 과거 피드백이 현재 쿼리 및 문서와 관련이 있습니까? (yes/no)
    """

    # LLM 호출: 출력의 일관성을 위해 temperature=0 사용
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  # 일관된 응답을 위해 temperature=0 설정
    )
    
    # 응답 텍스트를 추출 및 정규화하여 관련성 판단
    answer = response.choices[0].message.content.strip().lower()
    return 'yes' in answer  # 응답에 'yes'가 포함되어 있으면 True 반환

In [13]:
def adjust_relevance_scores(query, results, feedback_data):
    """
    과거 피드백을 기반으로 문서의 관련성 점수를 조정하여 검색 품질을 향상시킵니다.

    이 함수는 사용자 피드백을 분석하여, 현재 질의에 대한 검색 결과의 관련성 점수를 동적으로 조정합니다.
    관련 피드백을 식별하고, 평점 기반으로 점수 보정치를 계산한 후, 결과를 재정렬합니다.

    Args:
        query (str): 현재 사용자 질의
        results (List[Dict]): 검색된 문서들과 원래의 유사도 점수
        feedback_data (List[Dict]): 사용자 피드백 기록 (relevance 평점 포함)

    Returns:
        List[Dict]: 조정된 관련성 점수로 재정렬된 검색 결과 리스트
    """
    # 피드백이 없으면 원래 결과 그대로 반환
    if not feedback_data:
        return results

    print("피드백 기록을 기반으로 관련성 점수 조정 중...")

    # 각 검색된 문서에 대해 반복
    for i, result in enumerate(results):
        document_text = result["text"]
        relevant_feedback = []

        # 문서와 현재 질의 조합에 대해 관련 있는 피드백 탐색
        # LLM을 통해 각 피드백의 관련성 평가
        for feedback in feedback_data:
            is_relevant = assess_feedback_relevance(query, document_text, feedback)
            if is_relevant:
                relevant_feedback.append(feedback)

        # 관련 피드백이 있을 경우 점수 보정 적용
        if relevant_feedback:
            # 관련 피드백의 평균 관련성 평점 계산 (1~5 점수)
            avg_relevance = sum(f['relevance'] for f in relevant_feedback) / len(relevant_feedback)

            # 평균 평점을 0.5~1.5 사이의 보정 계수로 변환
            # - 3점 미만: 점수 감소 (modifier < 1.0)
            # - 3점 초과: 점수 증가 (modifier > 1.0)
            modifier = 0.5 + (avg_relevance / 5.0)

            # 원래 유사도 점수에 보정 계수 적용
            original_score = result["similarity"]
            adjusted_score = original_score * modifier

            # 결과 딕셔너리 업데이트
            result["original_similarity"] = original_score   # 원래 점수 저장
            result["similarity"] = adjusted_score            # 보정된 점수로 업데이트
            result["relevance_score"] = adjusted_score       # 관련성 점수도 업데이트
            result["feedback_applied"] = True                # 피드백 반영 여부 표시
            result["feedback_count"] = len(relevant_feedback)  # 사용된 피드백 수

            # 로그 출력
            print(
                f"  문서 {i+1}: {len(relevant_feedback)}개의 피드백을 기반으로 점수 {original_score:.4f} → {adjusted_score:.4f}로 조정됨 "
                f"(Document {i+1}: Adjusted score from {original_score:.4f} to {adjusted_score:.4f} based on {len(relevant_feedback)} feedback(s))"
            )

    # 보정된 점수 기준으로 결과 재정렬
    results.sort(key=lambda x: x["similarity"], reverse=True)

    return results

## Fine-tuning Our Index with Feedback

In [15]:
def fine_tune_index(current_store, chunks, feedback_data):
    """
    고품질 피드백을 활용하여 벡터 스토어를 향상시킵니다.

    이 함수는 다음 과정을 통해 벡터 스토어의 검색 품질을 지속적으로 개선합니다:
    1. 높은 평점을 받은 Q&A 쌍을 기반으로 고품질 피드백을 식별합니다.
    2. 성공적인 상호작용을 기반으로 새로운 검색 항목을 생성합니다.
    3. 이를 가중치를 높여 벡터 스토어에 추가합니다.

    Args:
        current_store (SimpleVectorStore): 원본 문서 조각들을 포함한 현재 벡터 스토어
        chunks (List[str]): 원본 문서의 텍스트 조각 리스트
        feedback_data (List[Dict]): 관련성과 품질 평점을 포함한 사용자 피드백 기록

    Returns:
        SimpleVectorStore: 원본 조각과 피드백 기반 콘텐츠를 포함한 향상된 벡터 스토어
    """
    print("고품질 피드백으로 인덱스를 미세 조정 중...")

    # 관련성 및 품질이 모두 4 이상인 고품질 피드백만 필터링합니다.
    # 가장 성공적인 상호작용만 학습에 사용하기 위함입니다.
    good_feedback = [f for f in feedback_data if f['relevance'] >= 4 and f['quality'] >= 4]

    if not good_feedback:
        print("미세 조정에 사용할 고품질 피드백이 없습니다.")
        return current_store  # 고품질 피드백이 없을 경우 기존 스토어 그대로 반환

    # 원본 및 향상된 콘텐츠를 모두 포함할 새로운 스토어를 초기화합니다.
    new_store = SimpleVectorStore()

    # 기존 문서 조각과 메타데이터를 새로운 스토어에 추가합니다.
    for i in range(len(current_store.texts)):
        new_store.add_item(
            text=current_store.texts[i],
            embedding=current_store.vectors[i],
            metadata=current_store.metadata[i].copy()  # 참조 문제를 피하기 위해 copy 사용
        )

    # 고품질 피드백을 기반으로 향상된 콘텐츠를 생성하고 추가합니다.
    for feedback in good_feedback:
        # 질문과 고품질 응답을 결합하여 새로운 문서를 생성합니다.
        # 이는 사용자의 질문에 직접 대응하는 검색 가능한 콘텐츠입니다.
        enhanced_text = f"Question: {feedback['query']}\nAnswer: {feedback['response']}"

        # 새로운 문서에 대해 임베딩 벡터를 생성합니다.
        embedding = create_embeddings(enhanced_text)

        # 피드백 기반 콘텐츠임을 나타내는 메타데이터와 함께 벡터 스토어에 추가합니다.
        new_store.add_item(
            text=enhanced_text,
            embedding=embedding,
            metadata={
                "type": "feedback_enhanced",     # 피드백 기반 콘텐츠로 표시
                "query": feedback["query"],      # 원본 질문 저장
                "relevance_score": 1.2,          # 검색 시 우선순위를 높이기 위한 가중치
                "feedback_count": 1,             # 피드백 반영 횟수
                "original_feedback": feedback    # 전체 피드백 기록 저장
            }
        )

        print(f"피드백에서 향상된 콘텐츠 추가됨: {feedback['query'][:50]}...")

    # 향상된 벡터 스토어의 항목 수를 출력합니다.
    print(f"미세 조정된 인덱스 항목 수: {len(new_store.texts)}개 (원본: {len(chunks)}개)")
    return new_store

## Complete RAG Pipeline with Feedback Loop

In [16]:
def generate_response(query, context, model="gpt-4o-mini"):
    """
    질의와 문맥을 기반으로 응답을 생성합니다.

    Args:
        query (str): 사용자 질의
        context (str): 검색된 문서에서 가져온 문맥 텍스트
        model (str): 사용할 LLM 모델 이름

    Returns:
        str: 생성된 응답 문자열
    """
    # LLM의 동작을 안내하는 시스템 프롬프트 정의
    system_prompt = """당신은 유용한 AI 비서입니다. 
    제공된 문맥에만 근거하여 사용자의 질문에 답변하세요. 
    문맥에서 답을 찾을 수 없는 경우 정보가 충분하지 않다고 말합니다."""
    
    # 문맥과 질의를 결합하여 사용자 프롬프트 생성
    user_prompt = f"""
        Context:
        {context}

        Question: {query}

        위의 문맥에만 근거하여 포괄적인 답변을 제공하세요.
    """
    
    # 시스템 및 사용자 프롬프트를 기반으로 OpenAI API를 호출하여 응답 생성
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0  # 일관된 결과를 위해 temperature=0 사용
    )
    
    # 생성된 응답 텍스트 반환
    return response.choices[0].message.content

In [17]:
def rag_with_feedback_loop(query, vector_store, feedback_data, k=5, model="gpt-4o-mini"):
    """
    피드백 루프가 통합된 RAG 파이프라인 전체 프로세스

    Args:
        query (str): 사용자 질의
        vector_store (SimpleVectorStore): 문서 청크가 저장된 벡터 저장소
        feedback_data (List[Dict]): 피드백 이력 데이터
        k (int): 검색할 문서 수
        model (str): 응답 생성을 위한 LLM 모델

    Returns:
        Dict: 질의, 검색된 문서, 생성된 응답을 포함한 결과 딕셔너리
    """
    print(f"\n***Processing query with feedback-enhanced RAG***")
    print(f"Query: {query}")

    # 1단계: 질의 임베딩 생성
    query_embedding = create_embeddings(query)

    # 2단계: 질의 임베딩을 기반으로 초기 문서 검색 수행
    results = vector_store.similarity_search(query_embedding, k=k)

    # 3단계: 검색된 문서들의 관련성 점수를 피드백 기반으로 조정
    adjusted_results = adjust_relevance_scores(query, results, feedback_data)

    # 4단계: 조정된 결과에서 텍스트만 추출하여 문맥 구성용 리스트 생성
    retrieved_texts = [result["text"] for result in adjusted_results]

    # 5단계: 검색된 텍스트들을 연결하여 응답 생성용 문맥 구성
    context = "\n\n---\n\n".join(retrieved_texts)

    # 6단계: 문맥과 질의를 기반으로 응답 생성
    print("Generating response...")
    response = generate_response(query, context, model)

    # 7단계: 최종 결과 딕셔너리 구성
    result = {
        "query": query,
        "retrieved_documents": adjusted_results,
        "response": response
    }

    print("\n***Response***")
    print(response)

    return result

## Complete Workflow: From Initial Setup to Feedback Collection

In [18]:
def full_rag_workflow(pdf_path, query, feedback_data=None, feedback_file="dataset/feedback_data.json", fine_tune=False):
    """
    피드백 통합을 통한 RAG 전체 워크플로우 실행

    이 함수는 Retrieval-Augmented Generation(RAG)의 전체 과정을 실행하며,
    피드백 기반 벡터 인덱스 향상과 사용자 피드백 수집을 포함합니다.

    주요 단계:
    1. 이전 피드백 데이터 로드
    2. 문서 처리 및 청크 분할
    3. 고품질 Q&A를 기반으로 벡터 인덱스 향상 (선택적)
    4. 피드백 기반 관련성 보정으로 검색 및 응답 생성
    5. 사용자 피드백 수집
    6. 피드백 저장 → 지속 학습 기반 마련

    Args:
        pdf_path (str): 처리할 PDF 문서 경로
        query (str): 사용자의 자연어 질의
        feedback_data (List[Dict], optional): 사전 로드된 피드백 데이터 (없으면 파일에서 불러옴)
        feedback_file (str): 피드백 이력을 저장할 JSON 파일 경로
        fine_tune (bool): 성공적인 Q&A를 벡터 인덱스에 반영할지 여부

    Returns:
        Dict: 생성된 응답 및 검색 관련 메타데이터를 포함한 결과
    """
    # 1단계: 피드백 데이터가 전달되지 않은 경우 파일에서 불러오기
    if feedback_data is None:
        feedback_data = load_feedback_data(feedback_file)
        print(f"{feedback_file}에서 {len(feedback_data)}개의 피드백 항목을 불러왔습니다.")

    # 2단계: 문서 처리 (텍스트 추출, 청크 분할, 임베딩 및 벡터 저장소 생성)
    chunks, vector_store = process_document(pdf_path)

    # 3단계: 이전 피드백을 활용하여 벡터 인덱스 개선 (fine_tune이 True일 때만)
    if fine_tune and feedback_data:
        vector_store = fine_tune_index(vector_store, chunks, feedback_data)

    # 4단계: 피드백 기반 RAG 실행 → 관련성 보정된 검색 + 응답 생성
    result = rag_with_feedback_loop(query, vector_store, feedback_data)

    # 5단계: 사용자 피드백 수집 (콘솔 입력)
    print("\n***이번 응답에 대한 피드백을 제공하시겠습니까?***")
    print("관련성 점수 (1~5, 5점이 가장 관련 있음):")
    relevance = input()

    print("품질 점수 (1~5, 5점이 가장 우수함):")
    quality = input()

    print("기타 코멘트가 있다면 입력해주세요 (건너뛰려면 Enter):")
    comments = input()

    # 6단계: 피드백 객체 구성
    feedback = get_user_feedback(
        query=query,
        response=result["response"],
        relevance=int(relevance),
        quality=int(quality),
        comments=comments
    )

    # 7단계: 피드백 저장 (지속적인 향후 학습을 위한 기록)
    store_feedback(feedback, feedback_file)
    print("피드백이 저장되었습니다. 감사합니다!")

    return result

## Evaluating Our Feedback Loop

In [19]:
def evaluate_feedback_loop(pdf_path, test_queries, reference_answers=None):
    """
    피드백 루프가 RAG 품질에 미치는 영향을 평가합니다.

    이 함수는 피드백 통합 전후의 성능을 비교하는 통제 실험을 수행합니다.
    다음과 같은 절차로 구성됩니다:
    1. 라운드 1: 피드백 없이 질의 실행 → 기준선(Baseline) 성능 측정
    2. 참조 답변이 있는 경우, 이를 기반으로 synthetic feedback 생성
    3. 라운드 2: 이전 피드백을 반영하여 동일 질의 재실행
    4. 결과를 비교하여 피드백 기반 개선의 정량적 효과 분석

    Args:
        pdf_path (str): 지식 베이스로 사용할 PDF 문서 경로
        test_queries (List[str]): 성능 평가용 테스트 질의 리스트
        reference_answers (List[str], optional): 정답 또는 참조 응답 리스트 (synthetic feedback 생성 및 비교용)

    Returns:
        Dict: 다음 정보를 포함하는 평가 결과
            - round1_results: 피드백 없이 실행한 응답 결과 목록
            - round2_results: 피드백을 반영한 응답 결과 목록
            - comparison: 두 라운드 간의 정량적 비교 결과
    """
    print("***피드백 루프 성능 평가 시작***")

    # 피드백을 임시 저장할 JSON 파일 경로 (세션 내 임시 용도)
    temp_feedback_file = "dataset/temp_evaluation_feedback.json"

    # 초기 피드백 리스트 (라운드 1에서는 없음)
    feedback_data = []

    # ----------------------- 라운드 1: 피드백 없이 -----------------------
    print("\n***라운드 1: 피드백 없음***")
    round1_results = []

    for i, query in enumerate(test_queries):
        print(f"\n질의 {i+1}: {query}")

        # 문서 처리 및 벡터 저장소 생성
        chunks, vector_store = process_document(pdf_path)

        # 피드백 없이 RAG 실행
        result = rag_with_feedback_loop(query, vector_store, [])
        round1_results.append(result)

        # 참조 답변이 있다면 → 이를 기반으로 synthetic feedback 생성
        if reference_answers and i < len(reference_answers):
            similarity_to_ref = calculate_similarity(result["response"], reference_answers[i])
            relevance = max(1, min(5, int(similarity_to_ref * 5)))
            quality = max(1, min(5, int(similarity_to_ref * 5)))

            feedback = get_user_feedback(
                query=query,
                response=result["response"],
                relevance=relevance,
                quality=quality,
                comments=f"참조 답변 유사도 기반 synthetic feedback: {similarity_to_ref:.2f}"
            )

            # 피드백을 메모리와 파일에 저장
            feedback_data.append(feedback)
            store_feedback(feedback, temp_feedback_file)

    # ----------------------- 라운드 2: 피드백 반영 -----------------------
    print("\n***라운드 2: 피드백 반영***")
    round2_results = []

    # 문서를 재처리하고 피드백 기반 인덱스 향상
    chunks, vector_store = process_document(pdf_path)
    vector_store = fine_tune_index(vector_store, chunks, feedback_data)

    for i, query in enumerate(test_queries):
        print(f"\n질의 {i+1}: {query}")

        # 피드백 반영된 RAG 실행
        result = rag_with_feedback_loop(query, vector_store, feedback_data)
        round2_results.append(result)

    # ----------------------- 결과 비교 -----------------------
    comparison = compare_results(test_queries, round1_results, round2_results, reference_answers)

    # 임시 피드백 파일 삭제 (실험 종료 후 정리)
    if os.path.exists(temp_feedback_file):
        os.remove(temp_feedback_file)

    return {
        "round1_results": round1_results,
        "round2_results": round2_results,
        "comparison": comparison
    }

## Helper Functions for Evaluation

In [20]:
def calculate_similarity(text1, text2):
    """
    두 텍스트 간 의미적 유사도를 임베딩 기반으로 계산합니다.

    Args:
        text1 (str): 첫 번째 텍스트
        text2 (str): 두 번째 텍스트

    Returns:
        float: 0과 1 사이의 유사도 점수 (1에 가까울수록 유사함)
    """
    # 두 텍스트에 대한 임베딩 생성
    embedding1 = create_embeddings(text1)
    embedding2 = create_embeddings(text2)
    
    # 임베딩을 NumPy 배열로 변환
    vec1 = np.array(embedding1)
    vec2 = np.array(embedding2)
    
    # 코사인 유사도 계산: 내적 / (벡터 크기의 곱)
    similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    
    return similarity

In [21]:
def compare_results(queries, round1_results, round2_results, reference_answers=None):
    """
    두 차례의 RAG 실행 결과를 비교 분석합니다.

    Args:
        queries (List[str]): 테스트 질의 목록
        round1_results (List[Dict]): 라운드 1 결과 (피드백 미사용)
        round2_results (List[Dict]): 라운드 2 결과 (피드백 반영)
        reference_answers (List[str], optional): 참조 정답 (있을 경우 평가에 포함)

    Returns:
        str: 각 질의별 비교 분석 결과 목록
    """
    print("\n=== COMPARING RESULTS ===")

    # 시스템 프롬프트: LLM에게 비교 기준을 안내
    system_prompt = """당신은 RAG 시스템의 전문 평가자입니다. 두 버전의 응답을 비교하세요:
    1. 표준 RAG: 피드백 사용 안 함
    2. 피드백 강화 RAG: 피드백 루프를 사용하여 검색을 개선합니다.

    어떤 버전이 다음과 같은 측면에서 더 나은 응답을 제공하는지 분석하세요:
    - 쿼리와의 관련성
    - 정보의 정확성
    - 완전성
    - 명확성 및 간결성
    """

    comparisons = []

    # 각 질의에 대해 두 라운드의 응답 비교
    for i, (query, r1, r2) in enumerate(zip(queries, round1_results, round2_results)):
        # 비교용 사용자 프롬프트 구성
        comparison_prompt = f"""
        Query: {query}

        Standard RAG Response:
        {r1["response"]}

        Feedback-enhanced RAG Response:
        {r2["response"]}
        """

        # 참조 정답이 있다면 포함
        if reference_answers and i < len(reference_answers):
            comparison_prompt += f"""
            Reference Answer:
            {reference_answers[i]}
            """

        comparison_prompt += """
        두 응답을 비교하고 어떤 응답이 더 나은지 그 이유를 설명하세요.
        특히 피드백 루프가 응답 품질을 어떻게 개선했는지(또는 개선하지 않았는지)에 초점을 맞춰 설명하세요.
        """

        # LLM을 통해 비교 분석 생성
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": comparison_prompt}
            ],
            temperature=0
        )

        # 분석 결과 저장
        comparisons.append({
            "query": query,
            "analysis": response.choices[0].message.content
        })

        # 분석 요약 출력
        print(f"\nQuery {i+1}: {query}")
        print(f"Analysis: {response.choices[0].message.content[:200]}...")

    return comparisons

## Evaluation of the feedback loop (Custom Validation Queries)

In [22]:
# AI 문서 경로 설정
pdf_path = "dataset/AI_Understanding.pdf"

# 테스트 질의 정의
test_queries = [
    "신경망이란 무엇이며 어떻게 작동하나요?",
    ### 테스트 속도를 위해 일부 질의는 주석 처리됨 ###
    
    # "Describe the process and applications of reinforcement learning.",
    # "What are the main applications of natural language processing in today's technology?",
    # "Explain the impact of overfitting in machine learning models and how it can be mitigated."
]

# 정답/참조 응답 정의
# (평가 및 synthetic feedback 생성을 위한 기준)
reference_answers = [
    "신경망은 인간의 뇌가 작동하는 방식을 모방한 프로세스를 통해 데이터 집합의 기본 관계를 인식하려는 일련의 알고리즘입니다. 신경망은 여러 층의 노드로 구성되며 각 노드는 뉴런을 나타냅니다. 신경망은 예상 결과와 비교한 출력의 오차에 따라 노드 간 연결의 가중치를 조정하여 작동합니다.",
    ### 테스트 속도를 위해 일부 정답 응답은 주석 처리됨 ###
    
    # "Reinforcement learning is a type of machine learning where an agent learns to make decisions by performing actions in an environment to maximize cumulative reward. It involves exploration, exploitation, and learning from the consequences of actions. Applications include robotics, game playing, and autonomous vehicles.",
    # "The main applications of natural language processing in today's technology include machine translation, sentiment analysis, chatbots, information retrieval, text summarization, and speech recognition. NLP enables machines to understand and generate human language, facilitating human-computer interaction.",
    # "Overfitting in machine learning models occurs when a model learns the training data too well, capturing noise and outliers. This results in poor generalization to new data, as the model performs well on training data but poorly on unseen data. Mitigation techniques include cross-validation, regularization, pruning, and using more training data."
]
# 평가 워크플로우 실행
# - RAG 성능을 피드백 전후로 비교
# - synthetic feedback 포함
evaluation_results = evaluate_feedback_loop(
    pdf_path=pdf_path,
    test_queries=test_queries,
    reference_answers=reference_answers
)

***피드백 루프 성능 평가 시작***

***라운드 1: 피드백 없음***

질의 1: 신경망이란 무엇이며 어떻게 작동하나요?
PDF에서 텍스트 추출 중... (Extracting text from PDF...)
텍스트 청크 분할 중... (Chunking text...)
21개의 텍스트 청크 생성됨 (Created 21 text chunks)
청크에 대한 임베딩 생성 중... (Creating embeddings for chunks...)
21개의 청크가 벡터 저장소에 추가됨 (Added 21 chunks to the vector store)

***Processing query with feedback-enhanced RAG***
Query: 신경망이란 무엇이며 어떻게 작동하나요?
Generating response...

***Response***
제공된 문맥에서는 신경망에 대한 구체적인 설명이 포함되어 있지 않습니다. 따라서 신경망이 무엇인지와 어떻게 작동하는지에 대한 정보가 충분하지 않습니다.

***라운드 2: 피드백 반영***
PDF에서 텍스트 추출 중... (Extracting text from PDF...)
텍스트 청크 분할 중... (Chunking text...)
21개의 텍스트 청크 생성됨 (Created 21 text chunks)
청크에 대한 임베딩 생성 중... (Creating embeddings for chunks...)
21개의 청크가 벡터 저장소에 추가됨 (Added 21 chunks to the vector store)
고품질 피드백으로 인덱스를 미세 조정 중...
미세 조정에 사용할 고품질 피드백이 없습니다.

질의 1: 신경망이란 무엇이며 어떻게 작동하나요?

***Processing query with feedback-enhanced RAG***
Query: 신경망이란 무엇이며 어떻게 작동하나요?
피드백 기록을 기반으로 관련성 점수 조정 중...
  문서 1: 1개의 피드백을 기반으로 점수 0.2880 → 0.2592

## Run Full RAG Workflow

In [23]:
# 대화형 예제 실행
print("\n\n***INTERACTIVE EXAMPLE***")
print("Enter your query about AI:")
user_query = input()

# 누적 피드백 로드
all_feedback = load_feedback_data()

# 전체 워크플로 실행
result = full_rag_workflow(
     pdf_path=pdf_path,
     query=user_query,
     feedback_data=all_feedback,
     fine_tune=True
)



***INTERACTIVE EXAMPLE***
Enter your query about AI:


 오늘날 AI 기술에서 자연어 처리의 주요 응용 분야는 무엇인가요?


피드백 데이터 파일이 존재하지 않습니다. 빈 리스트로 시작합니다.
PDF에서 텍스트 추출 중... (Extracting text from PDF...)
텍스트 청크 분할 중... (Chunking text...)
21개의 텍스트 청크 생성됨 (Created 21 text chunks)
청크에 대한 임베딩 생성 중... (Creating embeddings for chunks...)
21개의 청크가 벡터 저장소에 추가됨 (Added 21 chunks to the vector store)

***Processing query with feedback-enhanced RAG***
Query: 오늘날 AI 기술에서 자연어 처리의 주요 응용 분야는 무엇인가요?
Generating response...

***Response***
자연어 처리(NLP)의 주요 응용 분야는 다음과 같습니다: 

1. 챗봇: 인간과의 대화를 통해 정보를 제공하거나 문제를 해결하는 데 사용됩니다.
2. 기계 번역: 한 언어에서 다른 언어로 텍스트를 번역하는 데 활용됩니다.
3. 텍스트 요약: 긴 텍스트를 간결하게 요약하여 핵심 정보를 제공하는 데 사용됩니다.
4. 감정 분석: 텍스트에서 감정을 식별하고 분석하여 사용자 의견이나 감정을 이해하는 데 활용됩니다. 

이러한 응용 분야는 NLP 기술이 인간의 언어를 이해하고 해석하며 생성하는 데 중점을 두고 있음을 보여줍니다.

***이번 응답에 대한 피드백을 제공하시겠습니까?***
관련성 점수 (1~5, 5점이 가장 관련 있음):


KeyboardInterrupt: Interrupted by user

## Visualizing Feedback Impact

In [25]:
# 피드백 적용 전후 성능 비교 결과 추출
comparisons = evaluation_results['comparison']

# 분석 결과 출력
# 각 질의에 대해 피드백이 응답에 미친 영향 확인
print("\n피드백 효과 분석 결과\n")

for i, comparison in enumerate(comparisons):
    print(f"질의 {i+1}: {comparison['query']}")
    print("피드백 반영 효과 분석:")
    print(comparison['analysis'])
    print("\n" + "-"*50 + "\n")

# 응답 길이를 기반으로 충실도 간접 평가
# 라운드별 응답 길이 수집
round_responses = [evaluation_results[f'round{round_num}_results'] for round_num in range(1, len(evaluation_results) - 1)]
response_lengths = [[len(r["response"]) for r in round] for round in round_responses]

print("\n응답 길이 비교")
avg_lengths = [sum(lengths) / len(lengths) for lengths in response_lengths]

for round_num, avg_len in enumerate(avg_lengths, start=1):
    print(f"라운드 {round_num}: 평균 응답 길이 = {avg_len:.1f}자")

# 라운드 간 응답 길이 변화율 출력
if len(avg_lengths) > 1:
    changes = [
        (avg_lengths[i] - avg_lengths[i - 1]) / avg_lengths[i - 1] * 100
        for i in range(1, len(avg_lengths))
    ]
    for round_num, change in enumerate(changes, start=2):
        print(f"라운드 {round_num - 1}에서 라운드 {round_num}로의 길이 변화율: {change:.1f}%")


피드백 효과 분석 결과

질의 1: 신경망이란 무엇이며 어떻게 작동하나요?
피드백 반영 효과 분석:
두 버전의 응답을 비교해보면, 피드백 강화 RAG가 표준 RAG보다 더 나은 응답을 제공하지 못한 것으로 보입니다. 다음은 각 측면에서의 분석입니다:

1. **쿼리와의 관련성**:
   - **표준 RAG**: 신경망에 대한 구체적인 설명이 없다는 점을 지적하며, 쿼리에 대한 직접적인 답변을 제공하지 못했습니다.
   - **피드백 강화 RAG**: 동일하게 신경망에 대한 구체적인 설명이 없다는 점을 언급했지만, 추가적인 정보나 개선된 답변을 제공하지 않았습니다. 피드백 루프가 작동하지 않았습니다.

2. **정보의 정확성**:
   - 두 응답 모두 신경망에 대한 정보가 부족하다는 점에서 정확성은 동일하게 낮습니다. 피드백 루프가 작동하지 않아 정보의 정확성을 높이지 못했습니다.

3. **완전성**:
   - **표준 RAG**: 신경망에 대한 설명이 전혀 없기 때문에 완전성이 결여되어 있습니다.
   - **피드백 강화 RAG**: 마찬가지로 신경망에 대한 설명이 없으며, Reference Answer를 인용하지도 않았습니다. 따라서 완전성 역시 부족합니다.

4. **명확성 및 간결성**:
   - 두 응답 모두 명확성과 간결성에서 비슷한 수준입니다. 그러나 두 응답 모두 쿼리에 대한 직접적인 답변을 제공하지 않기 때문에 명확성은 떨어집니다. 피드백 루프가 추가적인 정보를 제공하지 않아 명확성을 높이지 못했습니다.

결론적으로, 피드백 강화 RAG는 표준 RAG와 비교했을 때 응답 품질을 개선하지 못했습니다. 피드백 루프가 작동하지 않아 신경망에 대한 구체적인 정보나 설명을 제공하지 못했으며, Reference Answer를 활용하지 않은 점이 특히 아쉬운 부분입니다. 따라서 두 응답 모두 부족하지만, 피드백 강화 RAG는 기대했던 개선 효과를 보여주지 못했습니다.

-----------------------------------------------